In [1]:
from importlib import reload
import pandas as pd
import time

In [2]:
import numpy as np
import spacy
from wordcloud import WordCloud


import matplotlib.pyplot as plt

In [3]:
import nltk
#nltk.download('stopwords')

In [4]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


In [5]:
import sklearn.feature_extraction.text as text

In [6]:
import nlp_pandas_functions as npf
#dir(npf)

2022-03-01 13:34:23,587 loading file /home/cdsw/.flair/models/sentiment-en-mix-distillbert_4.pt


In [7]:
#pip3 install pyarrow
#pip3 install spacy
#pip3 install wordcloud
#pip3 install nltk
#pip3 install sklearn
#pip3 install ipdb

In [8]:
str_state = 'GA'
file = 'agent_comments_sales_'+str_state
#file = 'agent_comments_testdf'
#READ THE SALES DATA - STATE
#df_sales = pd.read_feather(file + '_sentencecomposition.fea')
df_sales = pd.read_feather(file + '_withsentiment.fea')
#df_sales = pd.read_feather(file+'_processed.fea')


In [9]:
#df_sales_save = df_sales

In [10]:
#df_sales = df_sales_save#.head(10)

In [11]:
df_sales

,listingid,asgpropid,qtr,tax_year,cbsa_div,cbsa,state,Transdate,Transprice,bedrooms,...,clean_publicremarks_lemmatized_additional_words_textblobpolarity,publicremarks_vaderpolarity,clean_publicremarks_vaderpolarity,clean_publicremarks_additional_words_vaderpolarity,clean_publicremarks_nopunct_vaderpolarity,clean_publicremarks_nopunct_additional_words_vaderpolarity,clean_publicremarks_stemmed_vaderpolarity,clean_publicremarks_stemmed_additional_words_vaderpolarity,clean_publicremarks_lemmatized_vaderpolarity,clean_publicremarks_lemmatized_additional_words_vaderpolarity
0,226661582,27649366.0,202008,2020.0,23580,23580,GA,20200811,359000.0,4.0,...,"[0.2147619047619048, 0.4961904761904762]","{'compound': 0.9627, 'neg': 0.0, 'neu': 0.86, ...","{'compound': 0.9627, 'neg': 0.0, 'neu': 0.781,...","{'compound': 0.9401, 'neg': 0.0, 'neu': 0.803,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p...","{'compound': 0.6486, 'neg': 0.0, 'neu': 0.936,...","{'compound': 0.4939, 'neg': 0.0, 'neu': 0.954,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p..."
1,226786904,27628370.0,202008,2020.0,29300,29300,GA,20200925,265000.0,4.0,...,"[0.2828306878306879, 0.5542724867724869]","{'compound': 0.9939, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9898, 'neg': 0.013, 'neu': 0.70...","{'compound': 0.9876, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9889, 'neg': 0.014, 'neu': 0.69...","{'compound': 0.9864, 'neg': 0.016, 'neu': 0.7,...","{'compound': 0.9732, 'neg': 0.0, 'neu': 0.801,...","{'compound': 0.9686, 'neg': 0.0, 'neu': 0.799,...","{'compound': 0.9875, 'neg': 0.014, 'neu': 0.70...","{'compound': 0.9844, 'neg': 0.016, 'neu': 0.70..."
2,226142320,27963359.0,202005,2020.0,27600,27600,GA,20200624,239900.0,4.0,...,"[0.12000000000000002, 0.43]","{'compound': 0.8528, 'neg': 0.0, 'neu': 0.893,...","{'compound': 0.8221, 'neg': 0.0, 'neu': 0.853,...","{'compound': 0.8221, 'neg': 0.0, 'neu': 0.851,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.958,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.957,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,..."
3,226738459,27516267.0,202008,2020.0,29300,29300,GA,20200713,130900.0,3.0,...,"[0.0, 0.0]","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos..."
4,226850183,27791872.0,202008,2020.0,23580,23580,GA,20200702,406000.0,4.0,...,"[0.28250000000000003, 0.43]","{'compound': 0.8829, 'neg': 0.0, 'neu': 0.877,...","{'compound': 0.8829, 'neg': 0.0, 'neu': 0.849,...","{'compound': 0.8309, 'neg': 0.0, 'neu': 0.878,...","{'compound': 0.8658, 'neg': 0.0, 'neu': 0.857,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.886, ...","{'compound': 0.5106, 'neg': 0.0, 'neu': 0.932,...","{'compound': 0.2023, 'neg': 0.0, 'neu': 0.969,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.876, ...","{'compound': 0.6908, 'neg': 0.0, 'neu': 0.908,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633866,77898274,106321613.0,200708,2013.0,27600,27600,GA,20070719,163900.0,NaN,...,"[0.35000000000000003, 0.65]","{'compound': 0.7684, 'neg': 0.0, 'neu': 0.832,...","{'compound': 0.6351, 'neg': 0.0, 'neu': 0.834,...","{'compound': 0.6351, 'neg': 0.0, 'neu': 0.822,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.85, ...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.839,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.85, ...","{'compound': 0.5574, '

In [12]:
df_sales['transaction_year'] = np.floor(np.array((df_sales['Transdate'].values/10000)))

In [13]:
df_sales.columns

Index(['listingid', 'asgpropid', 'qtr', 'tax_year', 'cbsa_div', 'cbsa',
       'state', 'Transdate', 'Transprice', 'bedrooms', 'total_value',
       'cj_living_area', 'basement', 'has_pool', 'parking', 'is_poor',
       'is_good', 'HAS_VIEW', 'has_golf', 'has_water', 'has_woods', 'has_hill',
       'fips_code', 'census_tract', 'zip', 'effective_year_built', 'story',
       'ListingPrice', 'publicremarks', 'avmValue', 'avmerror', 'rentsale',
       'publicremarks_prepared', 'publicremarks_prepared_unicode',
       'clean_publicremarks', 'clean_publicremarks_additional_words',
       'clean_publicremarks_nopunct',
       'clean_publicremarks_nopunct_additional_words',
       'clean_publicremarks_stemmed',
       'clean_publicremarks_stemmed_additional_words',
       'clean_publicremarks_lemmatized',
       'clean_publicremarks_lemmatized_additional_words',
       'publicremarks_nltkpolarity', 'clean_publicremarks_nltkpolarity',
       'clean_publicremarks_additional_words_nltkpolarity',


In [14]:
print(df_sales['transaction_year'].min())
print(df_sales['transaction_year'].max())

2006.0
2021.0


In [15]:
df_sales_train = df_sales[df_sales['transaction_year']<=2020]

In [16]:
df_sales_train

,listingid,asgpropid,qtr,tax_year,cbsa_div,cbsa,state,Transdate,Transprice,bedrooms,...,publicremarks_vaderpolarity,clean_publicremarks_vaderpolarity,clean_publicremarks_additional_words_vaderpolarity,clean_publicremarks_nopunct_vaderpolarity,clean_publicremarks_nopunct_additional_words_vaderpolarity,clean_publicremarks_stemmed_vaderpolarity,clean_publicremarks_stemmed_additional_words_vaderpolarity,clean_publicremarks_lemmatized_vaderpolarity,clean_publicremarks_lemmatized_additional_words_vaderpolarity,transaction_year
0,226661582,27649366.0,202008,2020.0,23580,23580,GA,20200811,359000.0,4.0,...,"{'compound': 0.9627, 'neg': 0.0, 'neu': 0.86, ...","{'compound': 0.9627, 'neg': 0.0, 'neu': 0.781,...","{'compound': 0.9401, 'neg': 0.0, 'neu': 0.803,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p...","{'compound': 0.6486, 'neg': 0.0, 'neu': 0.936,...","{'compound': 0.4939, 'neg': 0.0, 'neu': 0.954,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p...",2020.0
1,226786904,27628370.0,202008,2020.0,29300,29300,GA,20200925,265000.0,4.0,...,"{'compound': 0.9939, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9898, 'neg': 0.013, 'neu': 0.70...","{'compound': 0.9876, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9889, 'neg': 0.014, 'neu': 0.69...","{'compound': 0.9864, 'neg': 0.016, 'neu': 0.7,...","{'compound': 0.9732, 'neg': 0.0, 'neu': 0.801,...","{'compound': 0.9686, 'neg': 0.0, 'neu': 0.799,...","{'compound': 0.9875, 'neg': 0.014, 'neu': 0.70...","{'compound': 0.9844, 'neg': 0.016, 'neu': 0.70...",2020.0
2,226142320,27963359.0,202005,2020.0,27600,27600,GA,20200624,239900.0,4.0,...,"{'compound': 0.8528, 'neg': 0.0, 'neu': 0.893,...","{'compound': 0.8221, 'neg': 0.0, 'neu': 0.853,...","{'compound': 0.8221, 'neg': 0.0, 'neu': 0.851,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.958,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.957,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,...",2020.0
3,226738459,27516267.0,202008,2020.0,29300,29300,GA,20200713,130900.0,3.0,...,"{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...",2020.0
4,226850183,27791872.0,202008,2020.0,23580,23580,GA,20200702,406000.0,4.0,...,"{'compound': 0.8829, 'neg': 0.0, 'neu': 0.877,...","{'compound': 0.8829, 'neg': 0.0, 'neu': 0.849,...","{'compound': 0.8309, 'neg': 0.0, 'neu': 0.878,...","{'compound': 0.8658, 'neg': 0.0, 'neu': 0.857,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.886, ...","{'compound': 0.5106, 'neg': 0.0, 'neu': 0.932,...","{'compound': 0.2023, 'neg': 0.0, 'neu': 0.969,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.876, ...","{'compound': 0.6908, 'neg': 0.0, 'neu': 0.908,...",2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633866,77898274,106321613.0,200708,2013.0,27600,27600,GA,20070719,163900.0,NaN,...,"{'compound': 0.7684, 'neg': 0.0, 'neu': 0.832,...","{'compound': 0.6351, 'neg': 0.0, 'neu': 0.834,...","{'compound': 0.6351, 'neg': 0.0, 'neu': 0.822,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.85, ...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.839,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.85, ...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.839,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.85, ...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.839,...",2007.0
633867,77863520,131534737.0,200705,2013.0,23580,23580,GA,20070

In [17]:
from nlp_pandas_functions import ngram_analysis
reload(ngram_analysis)
dir(ngram_analysis)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'bigrams_join',
 'count_n_grams',
 'defaultdict',
 'df_get_n_grams_count',
 'df_lambda_ngram',
 'generate_n_grams',
 'get_imp',
 'get_n_grams_count',
 'get_n_grams_probability',
 'kg',
 'ngrams',
 'np',
 'pd',
 'text']

In [18]:
column = 'clean_publicremarks_additional_words'

In [19]:
unigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_unigram.fea')
bigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_bigram.fea')
trigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_trigram.fea')
fourgram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_fourgram.fea')

In [20]:
df_sales_train[column].values[0]

'gorgeous basement desired braselton community riverstone park. entry, find formal dining room office complete french doors. open floor plan boasts kitchen large island & breakfast area overlooking living room open staircase DIGITnd floor. master main master suite upstairs walk-in closets bedroom! wind day screened porch gaze manicured lawn. meticulously cared lot offer, waiting you! riverstone park social neighborhood activities year. access life path easy golf cart ride favorite eateries!'

In [21]:
import time
from textblob import TextBlob
from collections import defaultdict

In [22]:
def get_n_grams_cumulative_error(df, column_name, ngram,
                            noun_sentences_dict_listingid_list,
                            noun_sentences_dict_error_list):
    
    tic = time.clock()
    #print((df[column_name].values))
    for idd,strsample in enumerate(df[column_name].values):
       
        textblob_temp = TextBlob(strsample)
        #print(idd, len(df[column_name].values),df['listingid'].values[idd])
        if len(textblob_temp.words)>=4:
            #print(strsample)
            #print(textblob_temp.words)
            df_ngram = ngram_analysis.get_n_grams_count(strsample,ngram)
            #print(df_ngram)
            #print(df_ngram)
            #print(df_ngram.empty)
            if df_ngram.empty==False: 
                df_ngram = df_ngram.rename({0: 'ngram', 1: 'count'}, axis='columns')
                df_ngram['error'] = df_ngram['count']*df['avmerror'].values[idd]
                df_ngram = df_ngram.set_index('ngram')

                error_temp = df_ngram.loc[list(df_ngram.index)]['error'].values
                #print(error_temp)
                #print(ngram_error.loc[list(df_ngram.index)]['error'].values)
                #new_error_list = list(ngram_error.loc[list(df_ngram.index)]['error'].values + 
                #                      error_temp)
                #print(new_error_list)
                #ngram_error.loc[list(df_ngram.index), ['error']] = new_error_list
                for idngram,strngram in enumerate(list(df_ngram.index)):
                    #print(strngram, error_temp[idngram],df['listingid'].values[idd])
                    temp_list_listing = noun_sentences_dict_listingid_list[strngram]
                    temp_list_listing.append(df['listingid'].values[idd])
                    temp_list_error = noun_sentences_dict_error_list[strngram]
                    temp_list_error.append(error_temp[idngram])

                    #print(temp_list_error)
                    noun_sentences_dict_listingid_list[strngram] = temp_list_listing
                    noun_sentences_dict_error_list[strngram] = temp_list_error
            if (idd%5000 == 0): 
                print(idd, len(df[column_name].values))
                print(time.clock() - tic)
            #if (idd%100 == 0): print(idd, len(df[column_name].values))

    return noun_sentences_dict_listingid_list, noun_sentences_dict_error_list

In [23]:
trigram_df_error = trigram_df
trigram_df_error = trigram_df_error.set_index('ngram')
noun_sentences_dict_listingid_list = dict(zip(list(trigram_df_error.index), 
                                              [[] for i in range(len(trigram_df['probability']))]))
noun_sentences_dict_error_list = dict(zip(list(trigram_df_error.index), 
                                          [[] for i in range(len(trigram_df['probability']))]))
trigram_df_error['error'] = trigram_df_error['probability']*0.
trigram_df_error.head(3)

,count,probability,error
ngram,,,
open floor plan,32978,0.001078,0.0
"stainless steel appliances,",22651,0.000741,0.0
rocking chair front,17785,0.000581,0.0


In [24]:
noun_sentences_dict_error_list

{'open floor plan': [],
 'stainless steel appliances,': [],
 'rocking chair front': [],
 'DIGIT bedroom DIGIT': [],
 'DIGIT car garage': [],
 'stainless steel appliances': [],
 'formal dining room': [],
 'granite counter tops,': [],
 'DIGIT bedroom, DIGIT': [],
 'DIGIT bedrooms DIGIT': [],
 'stainless steel appliances.': [],
 'formal dining room,': [],
 'hardwood floors throughout': [],
 'large master suite': [],
 'finished terrace level': [],
 'DIGIT sided brick': [],
 'large family room': [],
 'open floor plan,': [],
 'DIGIT bedrooms, DIGIT': [],
 'DIGIT car garage.': [],
 'bedroom DIGIT bath': [],
 'granite counter tops': [],
 'cul de sac': [],
 'separate dining room': [],
 'separate dining room,': [],
 'DIGIT car garage,': [],
 'chair front porch': [],
 'bedrooms DIGIT full': [],
 'open floor plan.': [],
 'formal living room': [],
 'hardwood floors main': [],
 'full finished basement': [],
 'large master bedroom': [],
 'bedroom, DIGIT bath': [],
 'bedroom full bath': [],
 'formal d

In [25]:
noun_sentences_dict_listingid_list, noun_sentences_dict_error_list  = get_n_grams_cumulative_error(df_sales_train, column, 3,
                            noun_sentences_dict_listingid_list,
                            noun_sentences_dict_error_list)


0 600278
0.024253000000015845
5000 600278
32.69153600000001
10000 600278
63.84903200000002
15000 600278
95.67319300000003
20000 600278
126.506115
25000 600278
159.339464
30000 600278
192.46084700000003
35000 600278
224.58933
40000 600278
255.223441
45000 600278
286.346453
50000 600278
318.62530000000004
55000 600278
350.01235900000006
60000 600278
381.03420800000004
65000 600278
411.843532
70000 600278
442.96671100000003
75000 600278
475.27741100000003
80000 600278
506.208984
85000 600278
538.14489
90000 600278
568.516881
95000 600278
599.1636
100000 600278
631.039078
105000 600278
662.809077
110000 600278
696.083247
115000 600278
729.813541
120000 600278
763.995457
125000 600278
798.527865
130000 600278
832.751501
135000 600278
867.068887
140000 600278
901.970769
145000 600278
936.754803
150000 600278
971.879529
155000 600278
1006.3193409999999
160000 600278
1041.23722
165000 600278
1076.642201
170000 600278
1111.442541
175000 600278
1146.286335
180000 600278
1181.426471
185000 600278

In [26]:
dict_keys = list(noun_sentences_dict_error_list.keys())

In [27]:
error_res2 = list(noun_sentences_dict_error_list.values()) 
listingid_res2 = list(noun_sentences_dict_listingid_list.values())

In [28]:
ngram_df = pd.DataFrame([dict_keys, error_res2, listingid_res2])

In [29]:
ngram_df = ngram_df.T

In [30]:
ngram_df = ngram_df.rename(columns={0: 'ngram', 1: 'AVM_Error_List', 
                                                        2: 'Listingid_List'})

In [31]:
ngram_df

,ngram,AVM_Error_List,Listingid_List
0,open floor plan,"[-0.13084281337047354, -0.04058923346986543, 0...","[226661582, 226754150, 226807226, 226857109, 2..."
1,"stainless steel appliances,","[0.06766127999999993, -0.03524390588235282, -0...","[226742674, 226871633, 226807295, 226759410, 2..."
2,rocking chair front,"[0.042625218841183865, 0.3281564, -0.015444285...","[226142320, 226934739, 226978566, 226916018, 2..."
3,DIGIT bedroom DIGIT,"[0.1408390151515152, -0.1755746753246754, -0.2...","[226095241, 226067653, 227258291, 226856227, 2..."
4,DIGIT car garage,"[0.09799119825708069, -0.17483815668202762, 0....","[227030658, 226661263, 226791999, 226979114, 2..."
...,...,...,...
13830166,ready! great bowen,[-0.032997567567567554],[77883696]
13830167,great bowen &,[-0.032997567567567554],[77883696]
13830168,& bowen resale,[-0.032997567567567554],[77883696]
13830169,bowen resale small,[-0.032997567567567554],[77883696]


In [32]:
trigram_df_error_ri = trigram_df_error.reset_index() 

In [33]:
trigram_df_error_merged = pd.merge(trigram_df_error_ri, ngram_df, on="ngram")

In [34]:
trigram_df_error_merged = trigram_df_error_merged.drop(columns=['error'])

In [35]:
from scipy.stats import moment
from scipy.stats import describe

def basic_stats_error(datalist):
    
    if len(datalist)>1:
        result_describe = list(describe(datalist)[2:])
        result_percentile = result_describe + list(np.percentile(datalist,[10,25,50,75,90]))
        array = np.array(datalist)
        result_percentile = result_percentile+[len(array),len(array[np.where(array>0)[0]])/len(array)*100,
                                               len(array[np.where(array<0)[0]])/len(array)*100]
    else:
        result_percentile = []

    return result_percentile

In [36]:
trigram_df_error_merged['AVM_Error_Indicators'] = trigram_df_error_merged['AVM_Error_List'].apply(lambda x: basic_stats_error(x))

In [37]:
trigram_df_error_merged

,ngram,count,probability,AVM_Error_List,Listingid_List,AVM_Error_Indicators
0,open floor plan,32978,1.078218e-03,"[-0.13084281337047354, -0.04058923346986543, 0...","[226661582, 226754150, 226807226, 226857109, 2...","[0.006261558393091166, 0.02461162458098134, -0..."
1,"stainless steel appliances,",22651,7.405763e-04,"[0.06766127999999993, -0.03524390588235282, -0...","[226742674, 226871633, 226807295, 226759410, 2...","[0.030043761539041428, 0.023683391745932095, 0..."
2,rocking chair front,17785,5.814820e-04,"[0.042625218841183865, 0.3281564, -0.015444285...","[226142320, 226934739, 226978566, 226916018, 2...","[0.006832851661749519, 0.029959435535577505, -..."
3,DIGIT bedroom DIGIT,17462,5.709215e-04,"[0.1408390151515152, -0.1755746753246754, -0.2...","[226095241, 226067653, 227258291, 226856227, 2...","[-0.014996603734997383, 0.044575033853082334, ..."
4,DIGIT car garage,15544,5.082124e-04,"[0.09799119825708069, -0.17483815668202762, 0....","[227030658, 226661263, 226791999, 226979114, 2...","[0.0010906490585247276, 0.03581556793750834, -..."
...,...,...,...,...,...,...
13830166,ready! great bowen,1,3.269508e-08,[-0.032997567567567554],[77883696],[]
13830167,great bowen &,1,3.269508e-08,[-0.032997567567567554],[77883696],[]
13830168,& bowen resale,1,3.269508e-08,[-0.032997567567567554],[77883696],[]
13830169,bowen resale small,1,3.269508e-08,[-0.032997567567567554],[77883696],[]


In [38]:
trigram_df_error_merged.to_feather('ngrams/' + file + '_withsentiment_erros_' + column + '_trigram.fea')

In [39]:
column

'clean_publicremarks_additional_words'

In [ ]:


for idd,ngramtemp in enumerate(trigram_df_error.index):
    if (idd%20000 == 0): print(idd, len(trigram_df_error))
    #print('     ',len(error_res2[idd]),np.max(error_res2[idd]))
    if (len((error_res2[idd]))>0):
        trigram_df_error.iloc[idd]['error_list'] = [[error_res2[idd]]]
        trigram_df_error.iloc[idd]['listingid_list'] = [[listingid_res2[idd]]]
 

In [ ]:
#81132 
#'illustrate potential.'


In [ ]:
listingid_res2[idd]

In [ ]:
dict_keys[idd]

In [ ]:
trigram_df_error.loc[0]

In [ ]:
trigram_df_error.iloc[0]['error_list']

In [ ]:
trigram_df_error

In [ ]:
len((res[0][1]))

In [ ]:
len((res2[0]))

In [ ]:
len((res[0][1]))

In [ ]:
np.array(noun_sentences_dict_error_list['family room']).shape

In [ ]:
d={'a':[1,2,3],'b':['a','b','c']}
list(d.values())[0]


In [ ]:
list(d.items())[0][1]

In [ ]:
list(noun_sentences_dict_error_list.items())[0]

In [ ]:
dict_values([[1, 2, 3], ['a', 'b', 'c']])
>>> list(d.values())
[[1, 2, 3], ['a', 'b', 'c']]


In [ ]:
len([np.array(noun_sentences_dict_error_list[ngramtemp])])

In [ ]:
trigram_df_error.loc[ngramtemp,'error_list'] = [[np.array(noun_sentences_dict_error_list[ngramtemp])]]


In [ ]:
trigram_df_error

In [ ]:
trigram_df_error['average_error'] = trigram_df_error['error'] / trigram_df_error['count']

In [ ]:
trigram_df_error = trigram_df_error.sort_values(by ='average_error', ascending=False)

In [ ]:
trigram_df_error

In [ ]:
trigram_df_error_filtered = trigram_df_error[trigram_df_error['count'] > 100]

In [ ]:
trigram_df_error_filtered.head(30)

In [ ]:
trigram_df_error_filtered.tail(30)

In [ ]:
trigram_df_error_ri = trigram_df_error.reset_index()

In [ ]:
trigram_df_error_ri

In [ ]:
trigram_df_error_ri.to_feather('ngrams/' + file + '_withsentiment_erros_' + column + '_trigram.fea')

In [ ]:
df_sales['avmerror']

In [ ]:
house = 5

In [ ]:
df_ngram = ngram_analysis.get_n_grams_count(df_sales[column].values[house],3)
df_ngram = df_ngram.rename({0: 'ngram', 1: 'count'}, axis='columns')
df_ngram['error'] = df_ngram['count']*df_sales['avmerror'].values[house]
df_ngram = df_ngram.set_index('ngram')
df_ngram

In [ ]:
list(df_ngram.index)

In [ ]:
list(trigram_df_error.loc[list(df_ngram.index)]['error'].values + df_ngram.loc[list(df_ngram.index)]['error'].values)

In [ ]:
new_error_list = list(trigram_df_error.loc[list(df_ngram.index)]['error'].values + df_ngram.loc[list(df_ngram.index)]['error'].values)
trigram_df_error.loc[list(df_ngram.index), ['error']] = new_error_list

In [ ]:
trigram_df_error.loc[list(df_ngram.index)] ['error']

In [ ]:
trigram_df_error.loc[list(df_ngram.index), ['error']] 

In [ ]:
list(trigram_df_error.loc[list(df_ngram.index)]['error'].values + df_ngram.loc[list(df_ngram.index)]['error'].values)

In [ ]:

trigram_df_error.loc[list(df_ngram.index)]['error']

In [ ]:
len(trigram_df_error.loc[list(df_ngram.index)]['error'])

In [ ]:
len(df_ngram.loc[list(df_ngram.index)]['error'])

In [ ]:
df_ngram.loc[list(df_ngram.index)]['error']

In [ ]:
trigram_df_error.sort_values(by ='error', ascending=False)